In [42]:
import cv2
import os
import time
import pandas as pd
import pyodbc
import datetime
import json

dataPath = '/home/nvidia/Documents/RF/Reconocimiento Facial/Data' #Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)

#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Leyendo el modelo
#face_recognizer.read('modeloEigenFace.xml')
#face_recognizer.read('modeloFisherFace.xml')
face_recognizer.read(dataPath + 'modeloLBPHFace.xml')

cap = cv2.VideoCapture(1)
#cap = cv2.VideoCapture('Video.mp4')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

# Try connect
cred =  open('cred.json',)
credentials = json.load(cred)
server = credentials['raiz']['server']
database = credentials['raiz']['database']
username = credentials['raiz']['username']
passw = credentials['raiz']['password']
driver= '/usr/lib/aarch64-linux-gnu/odbc/libtdsodbc.so'
table_name = 'RegistroAsistencia'

cnxn = pyodbc.connect(
    driver = driver,
    TDS_Version = '7.3', 
    server = server,
    port = 1433,
    DATABASE= 'AsistenciabSide',
    uid = username,
    pwd = passw)
cursor = cnxn.cursor()
registro = {'id_empleado': [], 'entrada':[]}

while True:
	ret,frame = cap.read()
	if ret == False: break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	auxFrame = gray.copy()

	faces = faceClassif.detectMultiScale(gray,1.3,5)

	for (x,y,w,h) in faces:
		rostro = auxFrame[y:y+h,x:x+w]
		rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
		result = face_recognizer.predict(rostro)

		cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
		'''
		# EigenFaces
		if result[1] < 5700:
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
		
		# FisherFace
		if result[1] < 500:
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
		'''

		# LBPHFace
		if result[1] < 75 :
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
			
			label = imagePaths[result[0]]
			if label in registro['id_empleado']:
				pass
			else:
				registro['id_empleado'].append(label)
				registro['entrada'].append(datetime.datetime.now())
				dfasis = pd.DataFrame.from_dict(registro)
				dfasis['entrada'] = pd.to_datetime(dfasis['entrada'])
				# Insert Dataframe into SQL Server:
				for index, row in dfasis.iterrows():
					cursor.execute("INSERT INTO dbo.RegistroAsistencia (id_empleado,entrada) values(?,?)", row.id_empleado, row.entrada)
				cnxn.commit()
				time.sleep(5)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

	cv2.imshow('frame',frame)
	k = cv2.waitKey(30)
	if  k & 0xff == ord('q'):
		cursor.close()
		break

cap.release()
cv2.destroyAllWindows()

imagePaths= ['15', '57', '38', '7', '37', '31', '51', '41', '18', '12', '10', '19', '34', '3', '48', '1', '33', '46', '42', '21', '32', '24', '6', '11', '13', '39', '35', '54', '20', '52', '17', '49', '16', '28', '55', '25', '40', '30', '5', '29', '9', '45', '4', '26', '47', '8', '50', '53', '36', '27', '2', '44', '22', '14', '23', '56', '43']


In [41]:
import pyodbc
import pandas as pd

driver = '/usr/lib/aarch64-linux-gnu/odbc/libtdsodbc.so' # Change this to where FreeTDS installed the driver library!


conn = pyodbc.connect(
    driver = driver,
    TDS_Version = '7.3', 
    server = server,
    port = 1433,
    DATABASE= 'AsistenciabSide',
    uid = username,
    pwd = passw)

crsr = conn.cursor()
table = pd.read_sql("""SELECT * FROM dbo.RegistroAsistencia """, conn)
#a =crsr.execute("SELECT name FROM sys.databases;")